In [39]:
# -*- coding: utf-8 -*
import twitter
import csv
import requests
from urllib2 import urlopen
import pandas as pd
from datetime import datetime, timedelta
from dateutil import tz
import numpy as np
import xml.etree.ElementTree as ET
import re

In [3]:
tree = ET.ElementTree(file=urlopen('https://roem.ru/rss/roem-all-news.xml'))
root = tree.getroot()
root.tag, root.attrib

('rss', {'version': '2.0'})

In [79]:
class RSSLoader:
    
    def __init__(self):
        #self._pages = ["https://roem.ru/rss/roem-all-news.xml", "http://lifenews.ru/xml/feed.xml", "http://www.forbes.ru/newrss.xml", "http://rg.ru/xml/index.xml", "http://www.vesti.ru/vesti.rss", "http://lenta.ru/rss", "http://ria.ru/export/rss2/index.xml"]
        self._pages = ["https://roem.ru/rss/roem-all-news.xml", "http://lifenews.ru/xml/feed.xml", "http://www.forbes.ru/newrss.xml", "http://www.vesti.ru/vesti.rss", "http://lenta.ru/rss", "http://ria.ru/export/rss2/index.xml", "https://navalny.com/blog/post.rss", "https://slon.ru/export/all.xml", "https://meduza.io/rss/all", "http://www.vedomosti.ru/rss/news"]
        self._month_dict = {"Jan":"1", "Feb":"2", "Mar":"3", "Apr":"4", "May":"5", "Jun":"6", "Jul":"7", "Aug":"8", "Sep":"9", "Oct":"10", "Nov":"11", "Dec":"12"}
        
        self._UTC_TIME_ZONE = tz.gettz('Europe/London')
        self._MOSCOW_TIME_ZONE = tz.gettz('Europe/Moscow')
        self._RATE_LIMIT = "[{u'message': u'Rate limit exceeded', u'code': 88}]"
        
        
    def _parse_date(self, date):

        """

        :param date: str, дата в формате - "Fri, 04 Dec 2015 14:45:39 +0000"
        :return: str, дата в человеческом, но буржуйском формате, да еще и в Московском часовом поясе
        """
        timezone = date.split("+")[1]
        date = date.replace(",", "")
        date_array = date.split(' ')
        
        time = date_array[4]
        day = int(date_array[1])
        month = self._month_dict[date_array[2]]
        year = date_array[3]

        date = str(year)+"-"+str(month)+"-"+str(day)+" "+time
        date = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
        
        if (timezone != "0300"):
            utc_date = date.replace(tzinfo=self._UTC_TIME_ZONE)
            date = utc_date.astimezone(self._MOSCOW_TIME_ZONE)

        # Обрезаем зону
        date = str(date).split("+")[0]
        
        # Обрезаем секунды
        splited = date.split(":")
        date = splited[0]+":"+splited[1]
        return date
    
    
    def _get_type(self, url):
        url = url.replace("www.", "")
        url = url.split("://")[1]
        url = url.split("/")[0]
        return url
    
    def _handle_data(self, url, link, title, date):
        
        # Убрать лишние пробелы
        title = re.sub(' +',' ', title)
        
        if url=="https://slon.ru/export/all.xml":
            link = link.split("?")[0]
            title = title.replace("\n", "")
            
        # Приведение даты к общему формату и временной зоне
        date = self._parse_date(date)
        
        return link, title, date
        
        
    
    def _get_hours_until_now(self, date):
        news_date = datetime.strptime(date, '%Y-%m-%d %H:%M')
        now = datetime.today()
        return float((now-news_date).total_seconds()/3600)
        
    def get_news_array(self, hour=5):
        """
        hour: int, время последней новости
        """
        news_array = []
        for url in self._pages:
            news_type = self._get_type(url)
            print url
            
            tree = ET.ElementTree(file=urlopen(url))
            root = tree.getroot()
            for i in root.iter('item'):

                link = i.find('link').text
                title = i.find('title').text
                date = i.find('pubDate').text
                
                link, title, date = self._handle_data(url, link, title, date)
                news_info =  {
                    "title": title,
                    "date": date, 
                    "url": link, 
                    "type": news_type
                }
                
                if self._get_hours_until_now(date) >= hour:
                    news_array.append(news_info)
                
            
        print "Собрано ", len(news_array), " новостей с RSS"
        return news_array

In [81]:
s = RSSLoader()
df = pd.DataFrame(s.get_news_array())

https://roem.ru/rss/roem-all-news.xml
http://lifenews.ru/xml/feed.xml
http://www.forbes.ru/newrss.xml
http://www.vesti.ru/vesti.rss
http://lenta.ru/rss
http://ria.ru/export/rss2/index.xml
https://navalny.com/blog/post.rss
https://slon.ru/export/all.xml
https://meduza.io/rss/all
http://www.vedomosti.ru/rss/news
Собрано  577  новостей с RSS


In [82]:
df

,date,title,type,url
0,2015-12-08 17:23,«Яндекс» с-acqhire-ил Agnitum — разработчика а...,roem.ru,https://roem.ru/08-12-2015/214617/yndx-agnitum/
1,2015-12-08 16:18,В Яндекс.Клавиатуре обнаружилась контекстная р...,roem.ru,https://roem.ru/08-12-2015/214605/yandex-advbo...
2,2015-12-08 14:53,Зампред комитета Годсдумы по науке подучил мол...,roem.ru,https://roem.ru/08-12-2015/214593/zampred-china/
3,2015-12-08 13:15,Блокировка RuTracker 9 декабря откладывается —...,roem.ru,https://roem.ru/08-12-2015/214569/peace-please/
4,2015-12-08 12:34,“У Рунета будет сильное и эффективное лобби. К...,roem.ru,https://roem.ru/08-12-2015/214560/kozlyuk-ny/
5,2015-12-08 10:34,Бренд Explay из расследования «ФАС и Яндекс пр...,roem.ru,https://roem.ru/08-12-2015/214539/explay-endplay/
6,2015-12-08 00:50,Кадры: Павел Пряников и десяток его единомышле...,roem.ru,https://roem.ru/08-12-2015/214528/lj-pryaniki-...
7,2015-12-07 22:38,Погиб основной владелец CDN Ngenix и совладеле...,roem.ru,https://roem.ru/07-12-2015/214517/dmitriy-shum...
8,2015-12-07 19:00,МТС запустит обещанный полтора года назад месс...,roem.ru,https://roem.ru/07-12-2015/214502/mtss-rcs/
9,2015-12-07 18:05,«Яндекс» отправит руководителей проектов общат...,roem.ru,https://roem.ru/07-12-2015/214499/ya-help/


In [84]:
df = df.append(s)

In [92]:
g=pd.DataFrame(df.groupby("url")["url"].count())
g.columns = ["url_count"]
g.reset_index(inplace=True)
df = df.merge(g, on="url", left_index=True, right_index=False)

In [100]:
len(df[df["url"]=="http://lenta.ru/news/2015/12/04/pantone/"])

1

In [30]:
for i in root.iter('item'):
    print i.find('link').text
    print i.find('title').text
    print i.find('pubDate').text

https://roem.ru/04-12-2015/214353/cubeline-streaming/
Как стриминг приложений в Google изменит рекламный рынок?
Fri, 04 Dec 2015 14:45:39 +0000
https://roem.ru/04-12-2015/214354/and-that-s-where-the-beginning-of-the-end-begun/
Говорят, на выходе из Яндекс.Новостей — все плохо, а «Ведомости» — не написавшее про Чайку гос-СМИ
Fri, 04 Dec 2015 14:29:33 +0000
https://roem.ru/04-12-2015/214351/ganin-yamail/
Кадры: Отвечавший за развитие Яндекс.Почты Егор Ганин пришел в Mail.ru Group
Fri, 04 Dec 2015 10:38:34 +0000
https://roem.ru/04-12-2015/214298/rbk-ny/
“Большой передел медиарынка, и меньший — телекома” — год глазами отраслевых журналистов РБК
Fri, 04 Dec 2015 09:52:01 +0000
https://roem.ru/04-12-2015/214334/app-play-nds/
Депутаты предложили обложить НДС покупки в App Store и Google Play
Fri, 04 Dec 2015 09:34:42 +0000
https://roem.ru/04-12-2015/214330/uber-dont-like-kids/
Открытка компании: Почему Uber не хочет возить детей?
Fri, 04 Dec 2015 08:45:50 +0000
https://roem.ru/04-12-2015/2143

In [ ]:
root = ET.fromstring(source.encode('utf-8'))#.getroot()

In [ ]:
return {
    "title": title, 
    "views": view, 
    "comments": comment, 
    "tags": tag_list, 
    "date": date, 
    "url": link, 
    "type": news_type, 
    "source": source
}

In [108]:
"Fri, 04 Dec 2015 14:45:39 +0000"

'Fri, 04 Dec 2015 14:45:39 +0000'

In [109]:
datetime.today().strftime("%a, %d %b %Y %I:%M:%S %z")

'Sat, 05 Dec 2015 05:08:41 '

In [118]:
#datetime.strftime("Fri, 04 Dec 2015 09:52:01 +0000", "%a, %d %b %Y %I:%M:%S %z")
datetime.strptime("Fri, 04 Dec 2015 09:52:01 +0000"[:-6], "%a, %d %b %Y %I:%M:%S")

datetime.datetime(2015, 12, 4, 9, 52, 1)